In [0]:
try:
    import prophet
    print("Prophet already installed.")
except ImportError:
    print("Prophet not found. Installing...")
    %pip install prophet

Prophet already installed.


In [0]:
# Imports

import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, sum as spark_sum, countDistinct, count
from prophet import Prophet
from prophet.plot import plot_plotly
from prophet.plot import plot_components_plotly
import plotly.express as px

## Dataset Exploration

In [0]:
# Loading Datasets

# Campaign Related
campaign_desc = spark.table("campaign_desc")
campaign_table = spark.table("campaign_table")

# Causal Data
casual_data = spark.table("causal_data")

# Coupon Related
coupon_info = spark.table("coupon")
coupon_redempt = spark.table("coupon_redempt")

# Demographic Related
hh_demographic = spark.table("hh_demographic")

# Product Related
product = spark.table("product")

# Transaction Related
transaction_data = spark.table("transaction_data")

In [0]:
# Function to get a basic idea of the table
def get_info(df):
    print("Number of rows: ", df.count())
    print("Number of columns: ", len(df.columns))
    print("Columns: ", df.columns)
    print("Schema: ", df.printSchema())
    df.show(5)
    print("Null + Blank Counts: \n", empty_counts(df))

# Function to get null counts
def empty_counts(df):
    return df.select([
        count(
            when(
                (col(c).isNull()) | (trim(col(c)) == ""), c
            )
        ).alias(c)
        for c in df.columns
    ]).toPandas().to_string(index=False)


In [0]:
# Campaign type and dates

get_info(campaign_desc)

Number of rows:  30
Number of columns:  4
Columns:  ['DESCRIPTION', 'CAMPAIGN', 'START_DAY', 'END_DAY']
root
 |-- DESCRIPTION: string (nullable = true)
 |-- CAMPAIGN: long (nullable = true)
 |-- START_DAY: long (nullable = true)
 |-- END_DAY: long (nullable = true)

Schema:  None
+-----------+--------+---------+-------+
|DESCRIPTION|CAMPAIGN|START_DAY|END_DAY|
+-----------+--------+---------+-------+
|      TypeB|      24|      659|    719|
|      TypeC|      15|      547|    708|
|      TypeB|      25|      659|    691|
|      TypeC|      20|      615|    685|
|      TypeB|      23|      646|    684|
+-----------+--------+---------+-------+
only showing top 5 rows
Null + Blank Counts: 
  DESCRIPTION  CAMPAIGN  START_DAY  END_DAY
           0         0          0        0


In [0]:
# household key, campaign, and description

get_info(campaign_table)

Number of rows:  7208
Number of columns:  3
Columns:  ['DESCRIPTION', 'household_key', 'CAMPAIGN']
root
 |-- DESCRIPTION: string (nullable = true)
 |-- household_key: long (nullable = true)
 |-- CAMPAIGN: long (nullable = true)

Schema:  None
+-----------+-------------+--------+
|DESCRIPTION|household_key|CAMPAIGN|
+-----------+-------------+--------+
|      TypeA|           17|      26|
|      TypeA|           27|      26|
|      TypeA|          212|      26|
|      TypeA|          208|      26|
|      TypeA|          192|      26|
+-----------+-------------+--------+
only showing top 5 rows
Null + Blank Counts: 
  DESCRIPTION  household_key  CAMPAIGN
           0              0         0


In [0]:
# Product, store, week, display, mailer

get_info(casual_data)

Number of rows:  36786524
Number of columns:  5
Columns:  ['PRODUCT_ID', 'STORE_ID', 'WEEK_NO', 'display', 'mailer']
root
 |-- PRODUCT_ID: long (nullable = true)
 |-- STORE_ID: long (nullable = true)
 |-- WEEK_NO: long (nullable = true)
 |-- display: string (nullable = true)
 |-- mailer: string (nullable = true)

Schema:  None
+----------+--------+-------+-------+------+
|PRODUCT_ID|STORE_ID|WEEK_NO|display|mailer|
+----------+--------+-------+-------+------+
|   9652583|     391|     14|      3|     0|
|   9652583|     391|     15|      3|     0|
|   9652583|     391|     21|      3|     0|
|   9652583|     391|     23|      3|     0|
|   9652583|     391|     24|      3|     0|
+----------+--------+-------+-------+------+
only showing top 5 rows
Null + Blank Counts: 
  PRODUCT_ID  STORE_ID  WEEK_NO  display  mailer
          0         0        0        0       0


In [0]:
# Coupon code, product, campaign

get_info(coupon_info)

Number of rows:  124548
Number of columns:  3
Columns:  ['COUPON_UPC', 'PRODUCT_ID', 'CAMPAIGN']
root
 |-- COUPON_UPC: long (nullable = true)
 |-- PRODUCT_ID: long (nullable = true)
 |-- CAMPAIGN: long (nullable = true)

Schema:  None
+-----------+----------+--------+
| COUPON_UPC|PRODUCT_ID|CAMPAIGN|
+-----------+----------+--------+
|10000089061|     27160|       4|
|10000089064|     27754|       9|
|10000089073|     28897|      12|
|51800009050|     28919|      28|
|52100000076|     28929|      25|
+-----------+----------+--------+
only showing top 5 rows
Null + Blank Counts: 
  COUPON_UPC  PRODUCT_ID  CAMPAIGN
          0           0         0


In [0]:
# household, redempt day, coupon code, campaign

get_info(coupon_redempt)

Number of rows:  2318
Number of columns:  4
Columns:  ['household_key', 'DAY', 'COUPON_UPC', 'CAMPAIGN']
root
 |-- household_key: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- COUPON_UPC: long (nullable = true)
 |-- CAMPAIGN: long (nullable = true)

Schema:  None
+-------------+---+-----------+--------+
|household_key|DAY| COUPON_UPC|CAMPAIGN|
+-------------+---+-----------+--------+
|            1|421|10000085364|       8|
|            1|421|51700010076|       8|
|            1|427|54200000033|       8|
|            1|597|10000085476|      18|
|            1|597|54200029176|      18|
+-------------+---+-----------+--------+
only showing top 5 rows
Null + Blank Counts: 
  household_key  DAY  COUPON_UPC  CAMPAIGN
             0    0           0         0


In [0]:
# Age, marital status, income, home owner

get_info(hh_demographic)

Number of rows:  801
Number of columns:  8
Columns:  ['AGE_DESC', 'MARITAL_STATUS_CODE', 'INCOME_DESC', 'HOMEOWNER_DESC', 'HH_COMP_DESC', 'HOUSEHOLD_SIZE_DESC', 'KID_CATEGORY_DESC', 'household_key']
root
 |-- AGE_DESC: string (nullable = true)
 |-- MARITAL_STATUS_CODE: string (nullable = true)
 |-- INCOME_DESC: string (nullable = true)
 |-- HOMEOWNER_DESC: string (nullable = true)
 |-- HH_COMP_DESC: string (nullable = true)
 |-- HOUSEHOLD_SIZE_DESC: string (nullable = true)
 |-- KID_CATEGORY_DESC: string (nullable = true)
 |-- household_key: long (nullable = true)

Schema:  None
+--------+-------------------+-----------+--------------+----------------+-------------------+-----------------+-------------+
|AGE_DESC|MARITAL_STATUS_CODE|INCOME_DESC|HOMEOWNER_DESC|    HH_COMP_DESC|HOUSEHOLD_SIZE_DESC|KID_CATEGORY_DESC|household_key|
+--------+-------------------+-----------+--------------+----------------+-------------------+-----------------+-------------+
|     65+|                  A|   

In [0]:
# Product, Manufacturer, Department, Brand, Commodity, Sub-Commodity, Size

get_info(product)

Number of rows:  92353
Number of columns:  7
Columns:  ['PRODUCT_ID', 'MANUFACTURER', 'DEPARTMENT', 'BRAND', 'COMMODITY_DESC', 'SUB_COMMODITY_DESC', 'CURR_SIZE_OF_PRODUCT']
root
 |-- PRODUCT_ID: long (nullable = true)
 |-- MANUFACTURER: long (nullable = true)
 |-- DEPARTMENT: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- COMMODITY_DESC: string (nullable = true)
 |-- SUB_COMMODITY_DESC: string (nullable = true)
 |-- CURR_SIZE_OF_PRODUCT: string (nullable = true)

Schema:  None
+----------+------------+------------+--------+--------------------+--------------------+--------------------+
|PRODUCT_ID|MANUFACTURER|  DEPARTMENT|   BRAND|      COMMODITY_DESC|  SUB_COMMODITY_DESC|CURR_SIZE_OF_PRODUCT|
+----------+------------+------------+--------+--------------------+--------------------+--------------------+
|     25671|           2|     GROCERY|National|            FRZN ICE| ICE - CRUSHED/CUBED|               22 LB|
|     26081|           2|MISC. TRANS.|National|NO COMM

In [0]:
# Information regarding Sales Value, Day, Product, Quantity, Store, Retail Disc, Trans Time, Week No, Coupon Disc, Coupon Match Disc

get_info(transaction_data)

Number of rows:  2595732
Number of columns:  12
Columns:  ['household_key', 'BASKET_ID', 'DAY', 'PRODUCT_ID', 'QUANTITY', 'SALES_VALUE', 'STORE_ID', 'RETAIL_DISC', 'TRANS_TIME', 'WEEK_NO', 'COUPON_DISC', 'COUPON_MATCH_DISC']
root
 |-- household_key: long (nullable = true)
 |-- BASKET_ID: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- PRODUCT_ID: long (nullable = true)
 |-- QUANTITY: long (nullable = true)
 |-- SALES_VALUE: double (nullable = true)
 |-- STORE_ID: long (nullable = true)
 |-- RETAIL_DISC: double (nullable = true)
 |-- TRANS_TIME: long (nullable = true)
 |-- WEEK_NO: long (nullable = true)
 |-- COUPON_DISC: double (nullable = true)
 |-- COUPON_MATCH_DISC: double (nullable = true)

Schema:  None
+-------------+-----------+---+----------+--------+-----------+--------+-----------+----------+-------+-----------+-----------------+
|household_key|  BASKET_ID|DAY|PRODUCT_ID|QUANTITY|SALES_VALUE|STORE_ID|RETAIL_DISC|TRANS_TIME|WEEK_NO|COUPON_DISC|COUPON_MATCH_DISC|
+

In [0]:
# Insights so far...

'''
    - We have transaction data at a household level along with campaign data and demographic data.
    
    - The campaign data is split into two tables, one with campaign description and the other with household key and campaign.
        - We will join these two tables to get campaign information for each household.

    - There are no null or blank values in any of the tables.
    
    - We define the scope of the project as follows
        - We will analyze the sales and discounts to look for anomalies
        - We will create useful visualization to understand the data better.
'''

'\n    - We have transaction data at a household level along with campaign data and demographic data.\n    \n    - The campaign data is split into two tables, one with campaign description and the other with household key and campaign.\n        - We will join these two tables to get campaign information for each household.\n\n    - There are no null or blank values in any of the tables.\n    \n    - We define the scope of the project as follows\n        - We will analyze the sales and discounts to look for anomalies\n        - We will create useful visualization to understand the data better.\n'

## TDQ/BDQ

In this section we will apply table specific checks to campaign, coupon and transaction data and come up with ways to handle data. Please note that in real-life scenarios all the tables have to go through these checks but for the scope of this project we are limiting to these 4 tables.

### Campaign Description Table

In [0]:
campaign_desc.show(5)

+-----------+--------+---------+-------+
|DESCRIPTION|CAMPAIGN|START_DAY|END_DAY|
+-----------+--------+---------+-------+
|      TypeB|      24|      659|    719|
|      TypeC|      15|      547|    708|
|      TypeB|      25|      659|    691|
|      TypeC|      20|      615|    685|
|      TypeB|      23|      646|    684|
+-----------+--------+---------+-------+
only showing top 5 rows


In [0]:
print("MIN CAMPAIGN VALUE:", campaign_desc.toPandas()['CAMPAIGN'].min())
print("Max CAMPAIGN VALUE:", campaign_desc.toPandas()['CAMPAIGN'].max())

print("DAY when first campaign was started:", campaign_desc.toPandas()['START_DAY'].min())
print("DAY when the last campaign ended:", campaign_desc.toPandas()['END_DAY'].max())

MIN CAMPAIGN VALUE: 1
Max CAMPAIGN VALUE: 30
DAY when first campaign was started: 224
DAY when the last campaign ended: 719


In [0]:
print("CAMPAIGN_DESC - TDQ + BDQ CHECKS")
print("="*60)

# 1. Uniqueness check on CAMPAIGN key
total = campaign_desc.count()
unique = campaign_desc.select(countDistinct("CAMPAIGN")).first()[0]
is_unique = (total == unique)
status = "PASSED" if is_unique else "FAILED - DUPLICATES FOUND!"

print(f"CAMPAIGN key uniqueness check => {status}")
print(f"   Total rows     : {total:,}")
print(f"   Unique values  : {unique:,}")
if not is_unique:
    print("   Duplicates present!")

# 2. BDQ: START_DAY <= END_DAY check
campaign_with_check = campaign_desc \
    .withColumn("day_sanity_flag", 
                when(col("START_DAY").cast("int") > col("END_DAY").cast("int"), 1)
                .otherwise(0))

invalid_count = campaign_with_check.select(spark_sum("day_sanity_flag")).first()[0]

print(f"\nSTART_DAY vs END_DAY check")
print(f"   Cases where START_DAY > END_DAY : {invalid_count}")
if invalid_count > 0:
    print("   START_DAY > END_DAY for some rows! => BDQ FAILURE")
    display(campaign_with_check.filter(col("day_sanity_flag") == 1)
            .select("CAMPAIGN", "DESCRIPTION", "START_DAY", "END_DAY"))
else:
    print("   START_DAY <= END_DAY for all rows! => PASSED")

# Drop the flag column
campaign_desc = campaign_with_check.drop("day_sanity_flag")

# Final verdict
if is_unique and invalid_count == 0:
    print(f"\nFINAL VERDICT: ALL CHECKS PASSED - CAMPAIGN_DESC IS CLEAN")
else:
    print(f"\nFINAL VERDICT: ISSUES FOUND - NEEDS ATTENTION")

CAMPAIGN_DESC - TDQ + BDQ CHECKS
CAMPAIGN key uniqueness check => PASSED
   Total rows     : 30
   Unique values  : 30

START_DAY vs END_DAY check
   Cases where START_DAY > END_DAY : 0
   START_DAY <= END_DAY for all rows! => PASSED

FINAL VERDICT: ALL CHECKS PASSED - CAMPAIGN_DESC IS CLEAN


In [0]:
# Hence unique on CAMPAIGN key level

### Campaign Table

In [0]:
campaign_table.show(5)

+-----------+-------------+--------+
|DESCRIPTION|household_key|CAMPAIGN|
+-----------+-------------+--------+
|      TypeA|           17|      26|
|      TypeA|           27|      26|
|      TypeA|          212|      26|
|      TypeA|          208|      26|
|      TypeA|          192|      26|
+-----------+-------------+--------+
only showing top 5 rows


In [0]:
print("MIN CAMPAIGN VALUE:", campaign_table.toPandas()['CAMPAIGN'].min())
print("Max CAMPAIGN VALUE:", campaign_table.toPandas()['CAMPAIGN'].max())

MIN CAMPAIGN VALUE: 1
Max CAMPAIGN VALUE: 30


In [0]:
df = campaign_table

total_rows = df.count()

# Check 1: Full composite key uniqueness (DESCRIPTION + household_key + CAMPAIGN)
composite_unique = df.select(countDistinct("DESCRIPTION", "household_key", "CAMPAIGN")).collect()[0][0]

# Check 2: Only (household_key + CAMPAIGN) uniqueness
household_campaign_unique = df.select(countDistinct("household_key", "CAMPAIGN")).collect()[0][0]

# Pretty print results
print("UNIQUE KEY ANALYSIS")
print("="*50)
print(f"Total rows: {total_rows:,}")

print(f"\n1. Unique on (DESCRIPTION + household_key + CAMPAIGN) => {composite_unique:,}")
print(f"   => {'PASSED' if composite_unique == total_rows else 'FAILED - DUPLICATES EXIST!'}")

print(f"\n2. Unique on (household_key + CAMPAIGN) => {household_campaign_unique:,}")
print(f"   => {'PASSED' if household_campaign_unique == total_rows else 'FAILED - SAME HOUSEHOLD GOT SAME CAMPAIGN MULTIPLE TIMES!'}")

# If household are duplicate then how many are duplicate
if household_campaign_unique < total_rows:
    duplicates = df.groupBy("household_key", "CAMPAIGN").count().filter("count > 1")
    affected_households = duplicates.count()
    print(f"\n   Affected (household + campaign) pairs : {affected_households}")
    print("   Sample duplicates:")
    display(duplicates.orderBy(col("count").desc()).limit(10))

UNIQUE KEY ANALYSIS
Total rows: 7,208

1. Unique on (DESCRIPTION + household_key + CAMPAIGN) => 7,208
   => PASSED

2. Unique on (household_key + CAMPAIGN) => 7,208
   => PASSED


### Coupon Redemption Table

In [0]:
coupon_redempt.show(5)

+-------------+---+-----------+--------+
|household_key|DAY| COUPON_UPC|CAMPAIGN|
+-------------+---+-----------+--------+
|            1|421|10000085364|       8|
|            1|421|51700010076|       8|
|            1|427|54200000033|       8|
|            1|597|10000085476|      18|
|            1|597|54200029176|      18|
+-------------+---+-----------+--------+
only showing top 5 rows


In [0]:
print("MIN CAMPAIGN VALUE:", coupon_redempt.toPandas()['CAMPAIGN'].min())
print("Max CAMPAIGN VALUE:", coupon_redempt.toPandas()['CAMPAIGN'].max())

print(coupon_redempt.toPandas()['DAY'].min())

MIN CAMPAIGN VALUE: 1
Max CAMPAIGN VALUE: 30
225


In [0]:
# Checking for duplicated rows

total_rows = coupon_redempt.count()
distinct_rows = coupon_redempt.select("household_key", "DAY", "CAMPAIGN", "COUPON_UPC").distinct().count()

print("Total rows:    ", total_rows)
print("Distinct rows: ", distinct_rows)

if total_rows == distinct_rows:
    print("\n Data IS at household_key x DAY x CAMPAIGN level x COUPON_UPC (no duplicates).")
else:
    print("\n Data is NOT unique at household_key x DAY x CAMPAIGN level x COUPON_UPC")

Total rows:     2318
Distinct rows:  2318

 Data IS at household_key x DAY x CAMPAIGN level x COUPON_UPC (no duplicates).


### Transaction Data

In [0]:
transaction_data.show(5)

+-------------+-----------+---+----------+--------+-----------+--------+-----------+----------+-------+-----------+-----------------+
|household_key|  BASKET_ID|DAY|PRODUCT_ID|QUANTITY|SALES_VALUE|STORE_ID|RETAIL_DISC|TRANS_TIME|WEEK_NO|COUPON_DISC|COUPON_MATCH_DISC|
+-------------+-----------+---+----------+--------+-----------+--------+-----------+----------+-------+-----------+-----------------+
|         2375|26984851472|  1|   1004906|       1|       1.39|     364|       -0.6|      1631|      1|        0.0|              0.0|
|         2375|26984851472|  1|   1033142|       1|       0.82|     364|        0.0|      1631|      1|        0.0|              0.0|
|         2375|26984851472|  1|   1036325|       1|       0.99|     364|       -0.3|      1631|      1|        0.0|              0.0|
|         2375|26984851472|  1|   1082185|       1|       1.21|     364|        0.0|      1631|      1|        0.0|              0.0|
|         2375|26984851472|  1|   8160430|       1|        1.5

In [0]:
# Sales_value is value after multiplying by quantity and substracting discount

transaction_data[(transaction_data['PRODUCT_ID'] == 1004906) & 
                 (transaction_data['QUANTITY'] == 1)].show(5)

+-------------+-----------+---+----------+--------+-----------+--------+-----------+----------+-------+-----------+-----------------+
|household_key|  BASKET_ID|DAY|PRODUCT_ID|QUANTITY|SALES_VALUE|STORE_ID|RETAIL_DISC|TRANS_TIME|WEEK_NO|COUPON_DISC|COUPON_MATCH_DISC|
+-------------+-----------+---+----------+--------+-----------+--------+-----------+----------+-------+-----------+-----------------+
|         2375|26984851472|  1|   1004906|       1|       1.39|     364|       -0.6|      1631|      1|        0.0|              0.0|
|         1929|27021022215|  4|   1004906|       1|       1.39|     441|       -0.6|      1755|      1|        0.0|              0.0|
|          568|27021090189|  4|   1004906|       1|       1.39|     446|       -0.6|        35|      1|        0.0|              0.0|
|          887|27031467254|  5|   1004906|       1|       1.39|     298|       -0.6|      1551|      1|        0.0|              0.0|
|          175|27057508267|  7|   1004906|       1|       1.99

In [0]:
print("Min Day:", transaction_data.toPandas()['DAY'].min())
print("Max Day:", transaction_data.toPandas()['DAY'].max())

Min Day: 1
Max Day: 711


In [0]:
"""
    Sanity Checks:
        - SALES_VALUE and QUANTITY should always be positive
        - DISCOUNT should always be negative or 0
        - The transaction data seems to be on a household_key x day x basket_id x product_id level. This has to be confirmed.
"""

'\n    Sanity Checks:\n        - SALES_VALUE and QUANTITY should always be positive\n        - DISCOUNT should always be negative or 0\n        - The transaction data seems to be on a household_key x day x basket_id x product_id level. This has to be confirmed.\n'

In [0]:
# High Level Check

condition = (
    (F.col("SALES_VALUE") < 0) |
    (F.col("QUANTITY") < 0) |
    (F.col("RETAIL_DISC") > 0) |
    (F.col("COUPON_DISC") > 0) |
    (F.col("COUPON_MATCH_DISC") > 0)
)

bad_count = transaction_data.filter(condition).count()

if bad_count > 0:
    print("ERROR: SALES_VALUE, QUANTITY, DISCOUNTS are not as expected")
else:
    print("SALES_VALUE, QUANTITY, DISCOUNTS are as expected")


ERROR: SALES_VALUE, QUANTITY, DISCOUNTS are not as expected


In [0]:
# Detailed Check

transaction_flagged = transaction_data \
    .withColumn("FLAG_SALES_VALUE", F.col("SALES_VALUE") < 0) \
    .withColumn("FLAG_QUANTITY",    F.col("QUANTITY") <= 0) \
    .withColumn("FLAG_RETAIL_DISC", F.col("RETAIL_DISC") > 0) \
    .withColumn("FLAG_COUPON_DISC", F.col("COUPON_DISC") > 0) \
    .withColumn("FLAG_MATCH_DISC",  F.col("COUPON_MATCH_DISC") > 0)

transaction_flagged.select(
    F.sum(F.col("FLAG_SALES_VALUE").cast("int")).alias("bad_sales_value"),
    F.sum(F.col("FLAG_QUANTITY").cast("int")).alias("bad_quantity"),
    F.sum(F.col("FLAG_RETAIL_DISC").cast("int")).alias("bad_retail_disc"),
    F.sum(F.col("FLAG_COUPON_DISC").cast("int")).alias("bad_coupon_disc"),
    F.sum(F.col("FLAG_MATCH_DISC").cast("int")).alias("bad_match_disc")
).show()


+---------------+------------+---------------+---------------+--------------+
|bad_sales_value|bad_quantity|bad_retail_disc|bad_coupon_disc|bad_match_disc|
+---------------+------------+---------------+---------------+--------------+
|              0|       14466|             36|              0|             0|
+---------------+------------+---------------+---------------+--------------+



In [0]:
# Displaying data with Retail Discount Anomaly

transaction_flagged.filter(
    F.col("FLAG_RETAIL_DISC")
).show(5, truncate=False)


+-------------+-----------+---+----------+--------+-----------+--------+------------+----------+-------+-----------+-----------------+----------------+-------------+----------------+----------------+---------------+
|household_key|BASKET_ID  |DAY|PRODUCT_ID|QUANTITY|SALES_VALUE|STORE_ID|RETAIL_DISC |TRANS_TIME|WEEK_NO|COUPON_DISC|COUPON_MATCH_DISC|FLAG_SALES_VALUE|FLAG_QUANTITY|FLAG_RETAIL_DISC|FLAG_COUPON_DISC|FLAG_MATCH_DISC|
+-------------+-----------+---+----------+--------+-----------+--------+------------+----------+-------+-----------+-----------------+----------------+-------------+----------------+----------------+---------------+
|1306         |31624096856|310|990941    |0       |0.01       |340     |0.01        |2253      |45     |0.0        |0.0              |false           |true         |true            |false           |false          |
|1306         |31735306143|318|7409789   |0       |0.0        |381     |2.220446E-16|1232      |46     |0.0        |0.0              |fa

In [0]:
# Displaying data with Quantity Anomaly

transaction_flagged.filter(
    F.col("FLAG_QUANTITY")
).show(5, truncate=False)


+-------------+-----------+---+----------+--------+-----------+--------+-----------+----------+-------+-----------+-----------------+----------------+-------------+----------------+----------------+---------------+
|household_key|BASKET_ID  |DAY|PRODUCT_ID|QUANTITY|SALES_VALUE|STORE_ID|RETAIL_DISC|TRANS_TIME|WEEK_NO|COUPON_DISC|COUPON_MATCH_DISC|FLAG_SALES_VALUE|FLAG_QUANTITY|FLAG_RETAIL_DISC|FLAG_COUPON_DISC|FLAG_MATCH_DISC|
+-------------+-----------+---+----------+--------+-----------+--------+-----------+----------+-------+-----------+-----------------+----------------+-------------+----------------+----------------+---------------+
|744          |26985165432|1  |5978648   |0       |0.0        |31582   |0.0        |1119      |1      |0.0        |0.0              |false           |true         |false           |false           |false          |
|1287         |26985336468|1  |5978648   |0       |0.0        |304     |0.0        |1351      |1      |0.0        |0.0              |false  

In [0]:
# Cleaning transaction data to remove transactions with anomalies

transaction_data = transaction_data.filter(
    (F.col("QUANTITY") != 0) &
    (F.col("RETAIL_DISC") <= 0)
)


In [0]:
# Running the high level check again

condition = (
    (F.col("SALES_VALUE") < 0) |
    (F.col("QUANTITY") < 0) |
    (F.col("RETAIL_DISC") > 0) |
    (F.col("COUPON_DISC") > 0) |
    (F.col("COUPON_MATCH_DISC") > 0)
)

bad_count = transaction_data.filter(condition).count()

if bad_count > 0:
    print("ERROR: SALES_VALUE, QUANTITY, DISCOUNTS are not as expected")
else:
    print("SALES_VALUE, QUANTITY, DISCOUNTS are as expected")


SALES_VALUE, QUANTITY, DISCOUNTS are as expected


In [0]:
# Checking the uniqueness level of the data

# 1. Total rows
total_rows = transaction_data.count()

# 2. Distinct rows at the expected granularity
distinct_rows = transaction_data.select(
    "HOUSEHOLD_KEY", "BASKET_ID", "DAY", "PRODUCT_ID"
).distinct().count()

print("Total rows:    ", total_rows)
print("Distinct grain:", distinct_rows)

if total_rows == distinct_rows:
    print("\n Data is at HOUSEHOLD_KEY x DAY x BASKET_ID x PRODUCT_ID level (no duplicates).")
else:
    print("\n Data is NOT at HOUSEHOLD_KEY x DAY x BASKET_ID x PRODUCT_ID level.")
    print("   There are duplicate rows for that combination.")


Total rows:     2581260
Distinct grain: 2581260

 Data is at HOUSEHOLD_KEY x DAY x BASKET_ID x PRODUCT_ID level (no duplicates).


In [0]:
# Insights so far...

"""
    - Both campaign_desc and campaign_table are ready to use.
    - The first campaign was launched on DAY 224 and the first redemption was seen on DAY 225 which checks out
    - We have 2 years worth of transaction data from Day-1 to Day-711.
    - Anomalies exists in transaction table with respect to RETAIL_DISC and QUANTITY.
        - The rows with the anomalies will be marked and reported to the relevant teams 
        - The rows are removed for further analysis.
    - Data is at HOUSEHOLD_KEY x DAY x BASKET_ID x PRODUCT_ID level (no duplicates).
"""

'\n    - Both campaign_desc and campaign_table are ready to use.\n    - The first campaign was launched on DAY 224 and the first redemption was seen on DAY 225 which checks out\n    - We have 2 years worth of transaction data from Day-1 to Day-711.\n    - Anomalies exists in transaction table with respect to RETAIL_DISC and QUANTITY.\n        - The rows with the anomalies will be marked and reported to the relevant teams \n        - The rows are removed for further analysis.\n    - Data is at HOUSEHOLD_KEY x DAY x BASKET_ID x PRODUCT_ID level (no duplicates).\n'

## Feature Engineering

In [0]:
# Aim of this exercise...

"""
    - We need a combined campaign table that maps household key to campaign as well start and end date of the campaign
        - This will help us better understand the customer behaviour during campaigns
    - We will create a daily transaction table for each customer to understand the customer behaviour during campaigns
    - Come up with a master table that can be used to understand the impact of campaigns on customer behaviour, we will have flags for campaign availability and redemption by the customers to track the impact of campaigns, awareness and redemption.
    - Create a daily level table summarising business level metrics such as sales, quantity, discounts etc.
"""

'\n    - We need a combined campaign table that maps household key to campaign as well start and end date of the campaign\n        - This will help us better understand the customer behaviour during campaigns\n    - We will create a daily transaction table for each customer to understand the customer behaviour during campaigns\n    - Come up with a master table that can be used to understand the impact of campaigns on customer behaviour, we will have flags for campaign availability and redemption by the customers to track the impact of campaigns, awareness and redemption.\n    - Create a daily level table summarising business level metrics such as sales, quantity, discounts etc.\n'

In [0]:
combined_campaign = (
    campaign_table.alias("t")
    .join(
        campaign_desc.select("CAMPAIGN", "START_DAY", "END_DAY").alias("d"),
        on="CAMPAIGN",
        how="left"        # keep all rows from campaign_table
    )
)

combined_campaign.show(20, truncate=False)

+--------+-----------+-------------+---------+-------+
|CAMPAIGN|DESCRIPTION|household_key|START_DAY|END_DAY|
+--------+-----------+-------------+---------+-------+
|26      |TypeA      |17           |224      |264    |
|26      |TypeA      |27           |224      |264    |
|26      |TypeA      |212          |224      |264    |
|26      |TypeA      |208          |224      |264    |
|26      |TypeA      |192          |224      |264    |
|26      |TypeA      |187          |224      |264    |
|26      |TypeA      |183          |224      |264    |
|26      |TypeA      |142          |224      |264    |
|26      |TypeA      |140          |224      |264    |
|26      |TypeA      |134          |224      |264    |
|26      |TypeA      |131          |224      |264    |
|26      |TypeA      |2492         |224      |264    |
|26      |TypeA      |2482         |224      |264    |
|26      |TypeA      |2478         |224      |264    |
|26      |TypeA      |2459         |224      |264    |
|26      |

In [0]:
print(campaign_table.count())
print(combined_campaign.count())

7208
7208


In [0]:
# Transaction aggregation table

transaction_daily = (
    transaction_data
    .groupBy("household_key", "DAY")
    .agg(
        F.sum("QUANTITY").alias("QUANTITY"),
        F.sum("SALES_VALUE").alias("SALES_VALUE"),
        F.sum("RETAIL_DISC").alias("RETAIL_DISC"),
        F.sum("COUPON_DISC").alias("COUPON_DISC")
    )
)

transaction_daily.show(10, truncate=False)


+-------------+---+--------+------------------+-------------------+-----------+
|household_key|DAY|QUANTITY|SALES_VALUE       |RETAIL_DISC        |COUPON_DISC|
+-------------+---+--------+------------------+-------------------+-----------+
|2141         |4  |18      |63.59999999999999 |-5.48              |0.0        |
|997          |36 |74      |198.13000000000005|-4.5               |0.0        |
|131          |41 |22      |64.51             |-2.99              |-0.5       |
|1914         |50 |27      |54.150000000000006|-1.72              |0.0        |
|2334         |65 |6       |19.11             |-3.71              |0.0        |
|1922         |66 |9       |30.509999999999998|-0.1               |0.0        |
|1707         |69 |4       |3.81              |-0.17              |0.0        |
|378          |72 |9       |17.43             |-1.08              |0.0        |
|716          |77 |5       |10.620000000000001|-2.08              |-0.35      |
|1463         |79 |33      |47.830000000

In [0]:
transaction_daily.count()

225344

In [0]:
t = transaction_data.alias("t")
c = combined_campaign.alias("c")

# Join only on matching household AND day in campaign window
joined = (
    t.join(
        c,
        (F.col("t.household_key") == F.col("c.household_key")) &
        (F.col("t.DAY") >= F.col("c.START_DAY")) &
        (F.col("t.DAY") <= F.col("c.END_DAY")),
        how="left"
    )
    # keep only the transaction fields + CAMPAIGN (drop duplicate household_key from c)
    .select(
        F.col("t.household_key").alias("household_key"),
        F.col("t.DAY").alias("DAY"),
        F.col("t.QUANTITY").alias("QUANTITY"),
        F.col("t.SALES_VALUE").alias("SALES_VALUE"),
        F.col("t.RETAIL_DISC").alias("RETAIL_DISC"),
        F.col("t.COUPON_DISC").alias("COUPON_DISC"),
        F.col("c.CAMPAIGN").alias("CAMPAIGN")
    )
)

# Collapse back to one row per household_key x DAY and create campaign_flag
transaction_daily = (
    joined
    .groupBy("household_key", "DAY")
    .agg(
        F.first("QUANTITY").alias("QUANTITY"),
        F.first("SALES_VALUE").alias("SALES_VALUE"),
        F.first("RETAIL_DISC").alias("RETAIL_DISC"),
        F.first("COUPON_DISC").alias("COUPON_DISC"),
        F.max(F.when(F.col("CAMPAIGN").isNotNull(), 1).otherwise(0)).alias("campaign_flag")
    )
)

transaction_daily.show(5, truncate=False)
transaction_daily.count()


+-------------+---+--------+-----------+-----------+-----------+-------------+
|household_key|DAY|QUANTITY|SALES_VALUE|RETAIL_DISC|COUPON_DISC|campaign_flag|
+-------------+---+--------+-----------+-----------+-----------+-------------+
|2141         |4  |1       |4.99       |-1.0       |0.0        |0            |
|997          |36 |1       |2.0        |-0.99      |0.0        |0            |
|131          |41 |1       |1.89       |-0.1       |0.0        |0            |
|1914         |50 |1       |2.99       |0.0        |0.0        |0            |
|2334         |65 |1       |4.17       |0.0        |0.0        |0            |
+-------------+---+--------+-----------+-----------+-----------+-------------+
only showing top 5 rows


225344

In [0]:
transaction_daily = (
    transaction_daily.alias("t")
    .join(
        # Collapse coupon_redempt to household_key x DAY with a flag
        coupon_redempt
            .select("household_key", "DAY")
            .dropDuplicates()
            .withColumn("coupon_redempt_flag", F.lit(1))
            .alias("c"),
        on=["household_key", "DAY"],
        how="left"
    )
    # Fill nulls with 0 for households/days with no redemption
    .withColumn(
        "coupon_redempt_flag",
        F.coalesce(F.col("coupon_redempt_flag"), F.lit(0))
    )
)

transaction_daily.count()
transaction_daily.show(20, truncate=False)


+-------------+---+--------+-----------+-----------+-----------+-------------+-------------------+
|household_key|DAY|QUANTITY|SALES_VALUE|RETAIL_DISC|COUPON_DISC|campaign_flag|coupon_redempt_flag|
+-------------+---+--------+-----------+-----------+-----------+-------------+-------------------+
|2141         |4  |1       |4.99       |-1.0       |0.0        |0            |0                  |
|997          |36 |1       |2.0        |-0.99      |0.0        |0            |0                  |
|131          |41 |1       |1.89       |-0.1       |0.0        |0            |0                  |
|1914         |50 |1       |2.99       |0.0        |0.0        |0            |0                  |
|2334         |65 |1       |4.17       |0.0        |0.0        |0            |0                  |
|1922         |66 |1       |7.59       |0.0        |0.0        |0            |0                  |
|1707         |69 |1       |1.52       |-0.17      |0.0        |0            |0                  |
|378      

In [0]:
transaction_daily.count()

225344

In [0]:
# Business Level Table

business_daily = (
    transaction_daily
    .groupBy("DAY")
    .agg(
        F.round(F.sum("SALES_VALUE"), 2).alias("total_sales"),
        F.round(F.sum("RETAIL_DISC"), 2).alias("total_retail_disc"),
        F.round(F.sum("COUPON_DISC"), 2).alias("total_coupon_disc"),

        # Household count metrics
        F.countDistinct(
            F.when(F.col("campaign_flag") == 1, F.col("household_key"))
        ).alias("campaign_count"),

        F.countDistinct(
            F.when(F.col("coupon_redempt_flag") == 1, F.col("household_key"))
        ).alias("coupon_redempt_count"),
    )
    .fillna(0)
    .orderBy("DAY")
)

display(business_daily)




DAY,total_sales,total_retail_disc,total_coupon_disc,campaign_count,coupon_redempt_count
1,24.01,-5.32,-1.0,0,0
2,133.96,-34.09,0.0,0,0
3,103.49,-19.63,-2.8,0,0
4,88.58,-11.95,0.0,0,0
5,88.4,-6.26,0.0,0,0
6,93.76,-23.33,0.0,0,0
7,57.62,-8.82,0.0,0,0
8,154.22,-34.8,0.0,0,0
9,90.89,-13.65,0.0,0,0
10,130.67,-28.17,0.0,0,0


In [0]:
# Development so far...

"""
    - We have built a combined campaign tables that maps every household to the campaign they were part of along with the days.
    - We have a daily level master transaction table with flags for campaign enrollment and coupon redemption.
    - We have a daily level business table with metrics for sales, retail discount, coupon discount, campaign enrollment and coupon redemption.
"""

'\n    - We have built a combined campaign tables that maps every household to the campaign they were part of along with the days.\n    - We have a daily level master transaction table with flags for campaign enrollment and coupon redemption.\n    - We have a daily level business table with metrics for sales, retail discount, coupon discount, campaign enrollment and coupon redemption.\n'

## Discripency Check

In [0]:
"""
  - We want to check if all the coupon discount is accounted for in the redemption table. Also, are all the coupon redemptions accounted for in the campaign table.
"""

'\n  - We want to check if all the coupon discount is accounted for in the redemption table. Also, are all the coupon redemptions accounted for in the campaign table.\n'

In [0]:
# Amount of unaccounted coupon discount

result = (
    transaction_daily
    .filter(
        (F.col("COUPON_DISC") < 0) &
        (F.col("coupon_redempt_flag") == 0)
    )
    .agg(F.sum("COUPON_DISC").alias("total_coupon_disc"))
    .collect()[0]["total_coupon_disc"]
)

print(result)


-1828.0399999999977


## ML Timeseries Anomaly Detection

In [0]:
"""
    We will detect anomalies in three criterions:
        - Sales
        - Retail Discount
        - Coupon Discount
"""

'\n    We will detect anomalies in three criterions:\n        - Sales\n        - Retail Discount\n        - Coupon Discount\n'

In [0]:
# Convert Spark daily table to Pandas
daily_pd = business_daily.select("DAY", "total_sales").orderBy("DAY").toPandas()

# Prophet requires columns: ds (datetime), y (value)

# Convert integer DAY to actual date
# Using 1960-01-01 origin (standard dunnhumby setup)
daily_pd["ds"] = pd.to_datetime(daily_pd["DAY"], unit="D", origin="1960-01-01")

### Sales

In [0]:
# Prophet target
daily_pd["y"] = daily_pd["total_sales"]

daily_pd = daily_pd[["ds", "y"]]
daily_pd.head()


,ds,y
0,1960-01-02,24.01
1,1960-01-03,133.96
2,1960-01-04,103.49
3,1960-01-05,88.58
4,1960-01-06,88.40


In [0]:
m_sales = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    interval_width=0.95
)

m_sales.fit(daily_pd)


08:19:26 - cmdstanpy - INFO - Chain [1] start processing
08:19:26 - cmdstanpy - INFO - Chain [1] done processing


In [0]:
# Forecast for historical + next 30 days
future_sales = m_sales.make_future_dataframe(periods=30)
forecast_sales = m_sales.predict(future_sales)

forecast_sales[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail()

,ds,yhat,yhat_lower,yhat_upper
736,1962-01-07,1736.285732,1301.707303,2163.036888
737,1962-01-08,1844.017377,1418.483165,2259.264799
738,1962-01-09,1859.334808,1455.132820,2302.979420
739,1962-01-10,1736.974099,1341.795588,2173.471222
740,1962-01-11,1715.668442,1310.755141,2107.311929


In [0]:
# Merge actual (y) with forecast bounds
merged = daily_pd.merge(
    forecast_sales[["ds", "yhat", "yhat_lower", "yhat_upper"]],
    on="ds",
    how="left"
)

# Condition: actual > upper bound or actual < lower bound => anomaly
merged["is_anomaly"] = (
    (merged["y"] > merged["yhat_upper"]) | 
    (merged["y"] < merged["yhat_lower"])
)


merged.head()


,ds,y,yhat,yhat_lower,yhat_upper,is_anomaly
0,1960-01-02,24.01,-6.571330,-408.590155,378.214726,False
1,1960-01-03,133.96,44.675175,-383.283470,470.319876,False
2,1960-01-04,103.49,160.312798,-258.807480,570.031913,False
3,1960-01-05,88.58,183.125579,-234.968330,564.619540,False
4,1960-01-06,88.40,67.907087,-370.422345,501.610856,False


In [0]:
# Extract anomaly dates
anomaly_sales_pd = merged[merged["is_anomaly"] == True][["ds", "y", "yhat_upper"]]

# Convert ds to DAY (integer) for joining back to Spark tables
anomaly_sales_pd["DAY"] = (anomaly_sales_pd["ds"] - pd.Timestamp("1960-01-01")).dt.days

anomaly_sales = spark.createDataFrame(anomaly_sales_pd[["DAY", "y", "yhat_upper"]])

anomaly_sales.show()

+---+-------+------------------+
|DAY|      y|        yhat_upper|
+---+-------+------------------+
|142|2043.21|1986.6207467337965|
|163|2274.54|2146.5764549987607|
|192|2249.96|2142.9041042880485|
|246|2296.01|1988.3964772815593|
|250|1163.03|2014.1997247520076|
|262|1256.74|2129.8919826395095|
|274|2123.09|2014.0166931378778|
|276|2306.31|2116.5700306826666|
|278|   5.59|2005.9738757962773|
|285|1087.12| 1978.045619226909|
|328|2087.01|1968.5435184439793|
|396|2482.58|2261.7144458354314|
|398|2169.64| 2109.678149241504|
|417|2475.66|2367.4037241489514|
|430| 2366.6|2365.6486458982763|
|437|2400.49|2328.1676382017363|
|504|2308.07| 2246.725802805291|
|507|1576.18| 2460.007627508798|
|508|1603.79| 2468.231158411286|
|509|1516.47|  2360.01617095989|
+---+-------+------------------+
only showing top 20 rows


In [0]:
fig = plot_plotly(m_sales, forecast_sales)
fig.update_layout(title="Daily Sales Forecast + Anomalies", height=600)
fig.show()

In [0]:
fig2 = plot_components_plotly(m_sales, forecast_sales)
fig2.update_layout(height=600)
fig2.show()

In [0]:
merged["anomaly_flag"] = (
    (merged["y"] > merged["yhat_upper"]) | 
    (merged["y"] < merged["yhat_lower"])
)

fig3 = px.scatter(
    merged,
    x="ds",
    y="y",
    color="anomaly_flag",
    color_discrete_map={False: "black", True: "red"},
    title="Sales Anomalies Highlighted",
    labels={"ds": "Date", "y": "Total Daily Sales"},
    height=500
)

fig3.show()

### Retail Discount

In [0]:
# Spark -> Pandas
retail_pd = (
    business_daily
    .select("DAY", "total_retail_disc", "campaign_count")
    .orderBy("DAY")
    .toPandas()
)

# Convert DAY -> date
retail_pd["ds"] = pd.to_datetime(retail_pd["DAY"], unit="D", origin="1960-01-01")

# Prophet expects positive values; retail_disc is negative, so take magnitude
retail_pd["y"] = retail_pd["total_retail_disc"].abs()

# Optional regressor: how many households in campaigns (explains normal discount spikes)
retail_pd["campaign_count"] = retail_pd["campaign_count"]

retail_pd = retail_pd[["ds", "y", "campaign_count"]]
retail_pd.head()

,ds,y,campaign_count
0,1960-01-02,5.32,0
1,1960-01-03,34.09,0
2,1960-01-04,19.63,0
3,1960-01-05,11.95,0
4,1960-01-06,6.26,0


In [0]:
m_retail = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    interval_width=0.95
)

# Add regressor
m_retail.add_regressor("campaign_count")

m_retail.fit(retail_pd)

08:19:31 - cmdstanpy - INFO - Chain [1] start processing
08:19:31 - cmdstanpy - INFO - Chain [1] done processing


In [0]:
# Future dataframe (historical + 30 days ahead)
future_retail = m_retail.make_future_dataframe(periods=30)

# Add regressor for future (set to last known value or 0 – you mainly care about history)
last_campaign = retail_pd["campaign_count"].iloc[-1]
future_retail["campaign_count"] = last_campaign

forecast_retail = m_retail.predict(future_retail)

# Merge actuals with forecast bounds
merged_retail = retail_pd.merge(
    forecast_retail[["ds", "yhat", "yhat_lower", "yhat_upper"]],
    on="ds",
    how="left"
)

# Flag BOTH high and low anomalies (magnitude too big or too small)
merged_retail["is_anomaly"] = (
    (merged_retail["y"] > merged_retail["yhat_upper"]) |
    (merged_retail["y"] < merged_retail["yhat_lower"])
)

In [0]:
fig_ret = plot_plotly(m_retail, forecast_retail)
fig_ret.update_layout(title="Daily Retail Discount Forecast + Actuals", height=600)
fig_ret.update_xaxes(rangeslider_visible=True)
fig_ret.show()

In [0]:
fig_ret_scatter = px.scatter(
    merged_retail,
    x="ds",
    y="y",
    color="is_anomaly",
    color_discrete_map={False: "white", True: "red"},
    title="Retail Discount Anomalies (High + Low)",
    labels={"ds": "Date", "y": "Daily |RETAIL_DISC|"},
    height=500
)
fig_ret_scatter.show()

In [0]:
retail_anom_pd = merged_retail[merged_retail["is_anomaly"] == True][["ds", "y", "yhat_upper", "yhat_lower"]]
retail_anom_pd["DAY"] = (retail_anom_pd["ds"] - pd.Timestamp("1960-01-01")).dt.days

retail_anomalies = spark.createDataFrame(retail_anom_pd[["DAY", "y", "yhat_upper", "yhat_lower"]])
retail_anomalies.show()

+---+------+------------------+------------------+
|DAY|     y|        yhat_upper|        yhat_lower|
+---+------+------------------+------------------+
|114|274.46|228.61869952866127|100.25434443494521|
|163|287.29|228.32831382680212|102.48260319196035|
|173|271.65|256.96092842360474|129.12477381371443|
|178| 246.1|239.17959929470646| 114.0302779878112|
|181| 117.4| 249.0946019045743| 122.7144159844371|
|196|240.31|238.60620211959989|107.74744397232658|
|245|326.13| 247.7299307656955|116.10296207003452|
|246|344.46|240.16144720878864|113.37251517680451|
|262| 98.16|240.09837757433868|111.16628577375913|
|276|281.62|240.76410960856037|110.91777382769827|
|277|289.23|260.68658301658513| 127.8893426435259|
|278|   0.0|257.91382264987527|130.01169119026096|
|319|272.14|263.55662960570027|140.55332638756312|
|377|277.79|240.93253642623893|120.65006020533642|
|413|241.96|240.09144330529315|113.12680159202009|
|422| 260.4|259.38093337830367| 131.8698313428723|
|433|271.45|264.36700463838855|

### Coupon Discount

In [0]:
coupon_pd = (
    business_daily
    .select("DAY", "total_coupon_disc", "campaign_count", "coupon_redempt_count")
    .orderBy("DAY")
    .toPandas()
)

# Convert DAY to a real date for Prophet
coupon_pd["ds"] = pd.to_datetime(coupon_pd["DAY"], unit="D", origin="1960-01-01")

# Prophet target: model magnitude of coupon discounts
coupon_pd["y"] = coupon_pd["total_coupon_disc"].abs()

# Regressors
coupon_pd["campaign_count"] = coupon_pd["campaign_count"]
coupon_pd["coupon_redempt_count"] = coupon_pd["coupon_redempt_count"]

coupon_pd = coupon_pd[["ds", "y", "campaign_count", "coupon_redempt_count"]]
coupon_pd.head()


,ds,y,campaign_count,coupon_redempt_count
0,1960-01-02,1.0,0,0
1,1960-01-03,0.0,0,0
2,1960-01-04,2.8,0,0
3,1960-01-05,0.0,0,0
4,1960-01-06,0.0,0,0


In [0]:
m_coupon = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    interval_width=0.95
)

# Add regressors to explain normal campaign-driven spikes
m_coupon.add_regressor("campaign_count")
m_coupon.add_regressor("coupon_redempt_count")

m_coupon.fit(coupon_pd)


08:19:36 - cmdstanpy - INFO - Chain [1] start processing
08:19:36 - cmdstanpy - INFO - Chain [1] done processing


In [0]:
future_coupon = m_coupon.make_future_dataframe(periods=30)

# Set regressor values for forecast period
future_coupon["campaign_count"] = coupon_pd["campaign_count"].iloc[-1]
future_coupon["coupon_redempt_count"] = coupon_pd["coupon_redempt_count"].iloc[-1]

# Predict
forecast_coupon = m_coupon.predict(future_coupon)

# Merge actuals with forecast bounds
merged_coupon = coupon_pd.merge(
    forecast_coupon[["ds", "yhat", "yhat_lower", "yhat_upper"]],
    on="ds",
    how="left"
)

# Detect BOTH high + low anomalies
merged_coupon["is_anomaly"] = (
    (merged_coupon["y"] > merged_coupon["yhat_upper"]) |
    (merged_coupon["y"] < merged_coupon["yhat_lower"])
)

In [0]:
fig_coupon = plot_plotly(m_coupon, forecast_coupon)
fig_coupon.update_layout(title="Daily Coupon Discount Forecast + Anomalies", height=600)
fig_coupon.update_xaxes(rangeslider_visible=False)
fig_coupon.show()

In [0]:
fig_coupon_scatter = px.scatter(
    merged_coupon,
    x="ds",
    y="y",
    color="is_anomaly",
    color_discrete_map={False: "red", True: "blue"},
    title="Coupon Discount Anomalies (High + Low)",
    labels={"ds": "Date", "y": "|COUPON_DISC|"},
    height=500
)

fig_coupon_scatter.show()

In [0]:
coupon_anom_pd = merged_coupon[merged_coupon["is_anomaly"] == True][["ds", "y", "yhat_upper", "yhat_lower"]]

# Convert ds to DAY integer
coupon_anom_pd["DAY"] = (coupon_anom_pd["ds"] - pd.Timestamp("1960-01-01")).dt.days

coupon_anomalies = spark.createDataFrame(
    coupon_anom_pd[["DAY", "y", "yhat_upper", "yhat_lower"]]
)

coupon_anomalies.show()

+---+-----+------------------+-------------------+
|DAY|    y|        yhat_upper|         yhat_lower|
+---+-----+------------------+-------------------+
| 90|18.58|  9.04190641042703|  -2.93451237065241|
| 92|19.33| 9.363418282326109| -2.847952391731113|
| 99|11.75| 9.981191427523271|-1.9211522464310495|
|128|20.74| 7.330061938284392|-4.6162858309513055|
|160|10.82| 8.952447597728558|-2.6914994625644146|
|167|10.45|  9.13913708221366| -3.394923598218247|
|189| 8.69| 8.422383992546093| -4.112236693279053|
|225|14.48| 8.590122266256534| -4.497745746511069|
|235|15.33| 9.104521246542355|-2.8365183069414748|
|289| 8.54| 8.113876168558075| -4.175303486297736|
|304|16.04| 8.499597366899014|-3.3517830160684112|
|319|13.29| 9.298244414190789| -3.613007277873772|
|324| 9.79| 7.926433796845469|-4.0544879519493335|
|366| 11.6| 8.169240924770392| -3.834101237987948|
|382|13.13| 8.978693538229008|-3.3091319205219474|
|390|11.85| 9.440206438521265| -3.060321846662676|
|404| 11.1| 8.402873528370302| 